In [1]:
from pyspark.sql import SparkSession
import findspark
from pyspark.conf import SparkConf
from delta import configure_spark_with_delta_pip
conf = SparkConf()
conf.set("spark.driver.maxResultSize", "8g")
conf.set("spark.driver.memory", "8g") 
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

builder = SparkSession.builder.appName("dev_test_notebook_2") \
    .enableHiveSupport()\
    .config(conf=conf)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-68b05c30-b2cd-46fe-8816-5c5bd6357b6b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 566ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

In [2]:
spark.sql("CREATE TABLE IF NOT EXISTS src_test2 (key INT, value STRING) USING hive")
spark.sql("LOAD DATA LOCAL INPATH '/opt/spark/examples/src/main/resources/kv1.txt' INTO TABLE src_test2")

23/11/03 13:02:49 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/03 13:02:49 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/03 13:02:53 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [3]:
spark.sql("SELECT * FROM src_test2").show()

23/11/03 13:02:54 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+---+-------+
|key|  value|
+---+-------+
|238|val_238|
| 86| val_86|
|311|val_311|
| 27| val_27|
|165|val_165|
|409|val_409|
|255|val_255|
|278|val_278|
| 98| val_98|
|484|val_484|
|265|val_265|
|193|val_193|
|401|val_401|
|150|val_150|
|273|val_273|
|224|val_224|
|369|val_369|
| 66| val_66|
|128|val_128|
|213|val_213|
+---+-------+
only showing top 20 rows



In [ ]:
spark.sql("SELECT COUNT(*) FROM src_test2").show()

In [ ]:
sqlDF = spark.sql("SELECT key, value FROM src_test2 WHERE key < 10 ORDER BY key")

In [ ]:
stringsDS = sqlDF.rdd.map(lambda row: "Key: %d, Value: %s" % (row.key, row.value))
for record in stringsDS.collect():
    print(record)

In [ ]:
from pyspark.sql import Row

Record = Row("key", "value")
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])
recordsDF.createOrReplaceTempView("records")

In [ ]:
spark.sql("SELECT * FROM records r JOIN src_test2 s ON r.key = s.key").show()

In [ ]:
spark.sql("SELECT * FROM default.cr_table_demo_delta").show()

In [ ]:
spark.stop()